# Light Therapy

I want to understand that actual power delivered by light therapy devices in the relevant wavelengths. Most (all?) manufacturers give very skant information. 

In [ ]:
# Working notebook
import pandas as pd
import numpy as np
import spectral

## General Notes on Light Therapy

Blue light is inteded to kill the bacteria that cause acne. The most effective wavelength is 415 nm +/- 5 nm with a power density of $30-50 mW/cm^2$. A total dose of $10-15 J/cm^2$. 

Red light is is absorbed by the cytochrome c oxidase in the skin mitochondria (fibroblasts) which stimulates the production of ATP, accelerating cellular repair and reducing inflammation. The most effective wavelength is 630 nm to 660 nm. From what I can tell, the same power density as blue light, $30-50 mW/cm^2$ is optimal. 

## Equipment

I'm using the HP330 spectrometer to calculate the power density spectrum of the light. As a calibration check, I'm using the Sekonic i-346. The i-346 meausres light power in lux, which is a measure of the light power perceived by human vision. To compare the readings, I adjust the actual light power spectrum by the lux human vision filter and then integrate. 

The HP330 and i-346 agree to within about 5%. 

## Filters

To avoid saturation I sometimes introduce filters before measuring with the HP330. To start, I'm just using white pieces of printer paper, which have a pretty strange absorption below about 450nm, so I'm calibrating the attenuation before applying it. The Filter 1, 2, and 3 curves are the filtration from 1, 2 and 3 stacked pieces of paper.  

In [6]:
# Calculate attenuation for filters 1, 2, and 3
filter_obj = spectral.Filter()
filter_dfs = []

for i in [1, 2, 3]:
    filter_df = filter_obj.calculate_attenuation(
        transmitted_file=f'data/{i}.csv',
        incident_file='data/0.csv'
    )
    filter_dfs.append(filter_df)

# Plot all filters using Analysis class
analysis = spectral.Analysis()
analysis.plot_multiple_attenuation(filter_dfs, labels=['Filter 1', 'Filter 2', 'Filter 3'])


In [7]:
# Calculate and apply Filter 1 to the spectrum
filter_obj = spectral.Filter()
filter_1 = filter_obj.calculate_attenuation(
    transmitted_file='data/1.csv',
    incident_file='data/0.csv'
)

# Apply Filter 1 to the spectrum
analysis = spectral.Analysis()
recovered_spectrum = analysis.remove_filter(df, filter_1)

# Display first few rows
print(f"Total data points: {len(recovered_spectrum)}")
print(f"Wavelength range: {recovered_spectrum['wavelength'].min()}nm - {recovered_spectrum['wavelength'].max()}nm")
print("\nFirst 10 rows of filtered spectrum:")
recovered_spectrum.head(10)


Total data points: 401
Wavelength range: 380nm - 780nm

First 10 rows of filtered spectrum:


,wavelength,intensity
0,380,0.022817
1,381,0.027080
2,382,0.023799
3,383,0.021350
4,384,0.021027
5,385,0.020616
6,386,0.020296
7,387,0.019461
8,388,0.016406
9,389,0.015241


# Revive Light Therapy

This is a plug-in unit that uses blue and red LEDs. Aside from using blue and red LED's, the manufacturer gives no specs. 

In [22]:
# Read revive.csv spectral data
hp330 = spectral.HP330()
spectrum = hp330.read_file('data/revive-1.csv')
spectrum = analysis.remove_filter(spectrum, filter_1) 
 
print(f"Revive spectrum loaded:")
print(f"  Total data points: {len(spectrum)}")
print(f"  Wavelength range: {spectrum['wavelength'].min()}nm - {spectrum['wavelength'].max()}nm")

analysis.plot_spectral(spectrum)

Revive spectrum loaded:
  Total data points: 401
  Wavelength range: 380nm - 780nm


In [23]:
# Calculate blue light power density and exposure time using new Analysis methods
blue_start = 410 
blue_end = 420  
target_energy = 15  # J/cm²

# Calculate power density
power_density_blue = analysis.calculate_power_density(
    spectrum,
    wavelength_start=blue_start,
    wavelength_end=blue_end,
    units='mW/cm2'
)

# Calculate exposure time
exposure_result = analysis.calculate_exposure_time(
    spectrum,
    wavelength_start=blue_start,
    wavelength_end=blue_end,
    target_energy=target_energy,
    units='J/cm2'
)
# Calculate lux (illuminance)
lux_value = analysis.power_to_lux(spectrum)

print(f"Blue light ({blue_start}-{blue_end}nm) power density: {power_density_blue:.6f} mW/cm²")
print(f"\nExposure time to reach {target_energy} {exposure_result['units']}:")
print(f"  {exposure_result['time_seconds']:.2f} seconds")
print(f"  {exposure_result['time_minutes']:.2f} minutes")
print(f"  Illuminance: {lux_value:.2f} lux")

Blue light (410-420nm) power density: 9.278844 mW/cm²

Exposure time to reach 15 J/cm2:
  1616.58 seconds
  26.94 minutes
  Illuminance: 26358.96 lux
